In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv("10k_diabetes/diab_train.csv")
df_test = pd.read_csv("10k_diabetes/diab_test.csv")
df_validate = pd.read_csv("10k_diabetes/diab_validation.csv")

In [5]:
df_train.head()

,Unnamed: 0,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted,diag_1_desc,diag_2_desc,diag_3_desc
0,9011,AfricanAmerican,Male,[60-70),?,Emergency,Discharged to home,Emergency Room,4,MC,...,No,No,No,No,No,No,0,Acute pericarditis in diseases classified else...,Secondary malignant neoplasm of kidney,"Congestive heart failure, unspecified"
1,9569,Caucasian,Female,[70-80),?,Elective,NaN,Physician Referral,1,?,...,No,No,No,No,Ch,Yes,1,Malignant essential hypertension,"Spinal stenosis, unspecified region",Diabetes mellitus without mention of complicat...
2,6090,Caucasian,Female,[80-90),?,Urgent,Discharged/transferred to SNF,Emergency Room,2,MC,...,No,No,No,No,Ch,Yes,0,"Urinary tract infection, site not specified",Streptococcus infection in conditions classifi...,"Congestive heart failure, unspecified"
3,4282,AfricanAmerican,Female,[50-60),?,Emergency,Discharged to home,Emergency Room,4,DM,...,No,No,No,No,Ch,Yes,1,"Respiratory abnormality, unspecified","Hypertensive chronic kidney disease, malignant...",Diabetes mellitus without mention of complicat...
4,9084,Caucasian,Male,[80-90),?,Elective,NaN,Physician Referral,13,?,...,No,No,No,No,Ch,Yes,1,Coronary atherosclerosis of unspecified type o...,"Chronic airway obstruction, not elsewhere clas...",Malignant essential hypertension
